In [50]:
import os
import re
import jieba 
import lda
import pandas as pd
import numpy as np

from collections import Counter, defaultdict

In [51]:
# 读取数据
data_path = '/Users/liuliangdong/project/jupyter_project/datasets/public_data/\
news_classfiy/train_new.csv'
label_path = '/Users/liuliangdong/project/jupyter_project/datasets/public_data/\
news_classfiy/label_new.csv'

datas = pd.read_csv(data_path, usecols=[1, 2], nrows=1000)
labels = pd.read_csv(label_path, usecols=[1, 2], nrows=1000)

In [52]:
# 1. 数据预处理，去除无关干扰信息
def replace_letter(str_):
    """ 正则去除英文字母 """
    return re.sub('[a-zA-Z]+.*[a-zA-Z]', '', str_)

datas['content'] = datas.apply(lambda x: replace_letter(x[1]), axis=1)

In [53]:
# 2. 分词并去除停用词
with open('../stopwords/中文停用词.txt', 'r', encoding='utf-8') as f:
    stopwords = list(map(lambda x: x.strip(), f.readlines()))
    
def cut_content(str_):
    return list(filter(
        lambda x: x not in stopwords and x.strip(),
        jieba.lcut(str_)
    ))

datas['content'] = datas.apply(lambda x: cut_content(x[1]), axis=1)

In [54]:
# 3. 对词语进行编号
def encode_id(series):
    """ 对所有词语进行统一编码 """
    d = defaultdict(int)
    n = 0
    for word_list in series:
        for word in word_list:
            if word not in d:
                d[word] = n
                n += 1
    return d
word_id = encode_id(datas.content)

id_word = {value: key for key, value in word_id.items()}

In [55]:
# 4. 统计词频
def count_word(word_list):
    word_count = Counter(word_list[1]) 
    return word_count

datas['count'] = datas.apply(count_word, axis=1)

In [56]:
datas.drop('content', axis=1, inplace=True)
datas.head(2)

,id,count
0,7a3dd79f90ee419da87190cff60f7a86,"{'几天': 1, '有人': 1, '举报': 5, '施': 2, '贴子': 3, '..."
1,7640a5589bc7486ca199eeeb38af79dd,"{'过去': 1, '一年': 2, '江歌': 15, '悲剧': 1, '几日': 1,..."


In [57]:
# 5. 将文本转换为矩阵 n * m;n:文档数，m:词数
def transform_(count_dict):
    result = []
    for w in word_id:
        result.append(count_dict.get(w, 0))
    return result

In [58]:
X = []
for i in range(datas.shape[0]):
    X.append(transform_(datas['count'][i]))
X = np.array(X)

In [59]:
Y = np.array(labels.label)

In [78]:
# 6. 构建模型训练数据
model = lda.LDA(50, random_state=9)    # 生成30个主题
model.fit(X)

INFO:lda:n_documents: 1000
INFO:lda:vocab_size: 37808
INFO:lda:n_words: 273591
INFO:lda:n_topics: 50
INFO:lda:n_iter: 2000
INFO:lda:<0> log likelihood: -4079853
INFO:lda:<10> log likelihood: -2748689
INFO:lda:<20> log likelihood: -2645730
INFO:lda:<30> log likelihood: -2602688
INFO:lda:<40> log likelihood: -2577701
INFO:lda:<50> log likelihood: -2557304
INFO:lda:<60> log likelihood: -2543954
INFO:lda:<70> log likelihood: -2534492
INFO:lda:<80> log likelihood: -2525136
INFO:lda:<90> log likelihood: -2516912
INFO:lda:<100> log likelihood: -2509653
INFO:lda:<110> log likelihood: -2502919
INFO:lda:<120> log likelihood: -2498995
INFO:lda:<130> log likelihood: -2495534
INFO:lda:<140> log likelihood: -2491527
INFO:lda:<150> log likelihood: -2488010
INFO:lda:<160> log likelihood: -2485284
INFO:lda:<170> log likelihood: -2483407
INFO:lda:<180> log likelihood: -2481572
INFO:lda:<190> log likelihood: -2479309
INFO:lda:<200> log likelihood: -2477491
INFO:lda:<210> log likelihood: -2476030
INFO:lda

In [79]:
# 主题词分布矩阵
topic_word = model.topic_word_
print('topic-words shape', topic_word.shape)
# 查看每篇文档的最重要的前10个词
for i in range(30):
    max_top10 = np.argsort(topic_word[i])[-20:][::-1]
    top_words = ' '.join(map(lambda x: id_word[x], max_top10))
    print(f'topic-{i + 1}主题词：', top_words)

topic-words shape (50, 37808)
topic-1主题词： 网友 妈妈 男子 @ 事件 应该 女子 道德 视频 这位 生命 面对 一名 情绪 熊 更 站 里 小女孩 更是
topic-2主题词： 三人 警方 南京 趙 恩 他們 赵恩 菏泽 後 祁剛 邮票 祁刚 菏澤 孙俪 樊軍傑 多年 郵票 樊军杰 1997 作案
topic-3主题词： 机场 城市 生态 长江 景洪 成长 西双版纳 中国 铁路 烟台 飞机 乘客 航班 泰国 成都 旅游 经济带 国际 景洪市 绑架
topic-4主题词： 记者 小区 供暖 11 公司 工作人员 居民 上午 供热 反映 今日 收费 15 安装 先生 市民 物业 了解 业主 用户
topic-5主题词： 元 赔偿 承担 责任 被告 南昌市 上诉人 原告 被上诉人 天 费用 进贤县 一审 规定 判决 本案 公安局 精神 计算 号
topic-6主题词： 派出所 宋某 宋 警察 烈士 英雄 笔录 王超 计恩武 没有 强奸 田晓花 提起 诉讼 身份 情况 镇安县 郭娟 妮 身份证
topic-7主题词： 诈骗 骗 元 骗子 微信 商品 受害人 微商 朋友圈 19 16 推销 已经 警方 购买 代理商 对象 电话 联系 冒充
topic-8主题词： 安全 事故 生产 发生 2018 责任 造成 死亡 检查 考试 工作 有限公司 公务员 人员 外卖 落实 开展 导致 现场 单位
topic-9主题词： 规定 行为 诈骗 应当 刑法 情形 犯罪 处罚 罪 死亡 认定 具有 财物 解释 构成 伪造 诈骗罪 情节 刘 法院
topic-10主题词： 工程 合同 施工 公司 建设 约定 工程款 承包人 当事人 工程施工 案 陈月勤 承包 旭日 建设工程 结算 价款 铁塔 没有 有限公司
topic-11主题词： 企业 数据 造假 发展 今年 监测 环保 我家 万人 家 全国 练江 产业 污染 重点 主要 产能 2018 市场 环境监测
topic-12主题词： 消费者 公司 元 广告 记者 商家 产品 食品 销售 购买 相关 投诉 女士 消费 店家 1000 价格 万元 进行 生产
topic-13主题词： 关注 点击 请 来源 公众 更 阅读 信息 生活 小编 图片 原文 事 | 号 编辑 长 网友 文章 新闻
topic-14

In [80]:
# 查看每篇文档属于每个主题的概率
doc_topics = model.doc_topic_
# 查看前5篇文档所属主题
for i in range(10):
    print(f'第{i+1}篇文档属于主题-{np.argmax(doc_topics[i])}')

第1篇文档属于主题-34
第2篇文档属于主题-39
第3篇文档属于主题-49
第4篇文档属于主题-17
第5篇文档属于主题-17
第6篇文档属于主题-17
第7篇文档属于主题-49
第8篇文档属于主题-22
第9篇文档属于主题-33
第10篇文档属于主题-3


## 使用kmeans对文档进行聚类,按聚类进过进行分类

In [69]:
from sklearn.cluster import KMeans

In [72]:
clf = KMeans(n_clusters=3, random_state=9)
y_hat = clf.fit_predict(doc_topics)

In [81]:
clf = KMeans(n_clusters=3, random_state=9)
y_hat1 = clf.fit_predict(doc_topics)

In [82]:
np.mean(y_hat1 == Y)

0.576

## 对未知数据进行分析 

In [85]:
test_path = '/Users/liuliangdong/project/jupyter_project/datasets/public_data/\
news_classfiy/Test_DataSet.csv'
test_data = pd.read_csv(test_path, nrows=100)